In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# API 키 가져오기
MY_OPENAI_KEY = os.getenv("MY_OPENAI_KEY")

client = OpenAI(api_key=MY_OPENAI_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


# 데이터 전처리 함수 작성 241210

### 리뷰 JSON 파일 로드

In [9]:
import json

with open('./resources/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[: 3]

[{'review': '서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍',
  'stars': 4,
  'date': '2024.01.24'},
 {'review': '인사동점 이용해보고 너무 좋아서 용산점 이용했습니다~\n건물도 너무 깨끗하고 직원들도 너무 친절했어요~\n게다가 무인락커가 있어서 락커에 짐보관이 너무 편하고좋았습니다~^^',
  'stars': 5,
  'date': '6일 전'},
 {'review': '친구가 생일이라서 다 같이 방문을 했는데 1

#### 좋은 평점 및 나쁜 평점 정의
 - 좋은 평점 = 별 5개
 - 나쁜 평점 = 별 4개 이하

In [10]:
good_count, bad_count = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_count += 1
    else:
        bad_count += 1

good_count, bad_count

(79, 21)

### 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [11]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가

reviews_bad[:3]

['[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]',
 '[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네

In [13]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:1000]

'[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]\n[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네요.\n

### 전처리 함수 작성

In [15]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./resources/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date # 어차피 6개월 안에 들어와서..

        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:500], bad[:500]

('[REVIEW_START]인사동점 이용해보고 너무 좋아서 용산점 이용했습니다~\n건물도 너무 깨끗하고 직원들도 너무 친절했어요~\n게다가 무인락커가 있어서 락커에 짐보관이 너무 편하고좋았습니다~^^[REVIEW_END]\n[REVIEW_START]수영장이 이쁘고 사진찍기 좋아요. 친절하셔서 좋앗어요[REVIEW_END]\n[REVIEW_START]체크인할때 프론트 여자직원이 정말 친절해서 기억에 남아요. 서울에 또 방문할때 여기 숙소가 1순위입니다. 감사합니다[REVIEW_END]\n[REVIEW_START]깔끔하게 쉬다 옴니다요[REVIEW_END]\n[REVIEW_START]용산역과 가깝고 주변에 아이파크몰도 있어서\n친구들과 하루 날잡아 쉬기 좋았어요.\n베개높이에 민감한데 적당하고 샤워실과 화장실 세면대가 각각 따로 있어서 여자3명이 머물기 편했습니다.\n다음엔 못온 친구와 4인실 이용해보고 싶네요.[REVIEW_END]\n[REVIEW_START]한강뷰가 잘 안보인다는 후기가 있어 걱정했는데',
 '[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네요.\n음식 자체가 퀄리티가 좋아서 또 갈 예정입니다![REVIEW_END]\n[REVIEW_START]3인 이그제큐티브로 이용했는데 티타임, 주류무제한 해피아워, 뷔페식 조식이 다포함이 된 가격이라 넘 좋았습니다!\n객실도 깨끗하고 직원분들도 친절하고, 무엇보다 유료인 수영장이 무료로 포함이 되어 있는 점이 좋았습니다.\n서울에 가본 이그제큐티브 중에서도 제일 음식 퀄리티가 높아서 2번째 방문이고 다음에도 또 갈 계획 있습니다![REVIEW_END]\n[REVIEW_START]깨끗하고 좋앗어요

### 평가용 함수 작성

#### 평가 기준 설정
- MT-Bench 논문 기반 Pariwise Comparison (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수 라는게 애매할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편!

#### 평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accomodations)

In [22]:
def pairwise_eval(reviews, answer_a, answer_b):
        eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
        
        completion = client.chat.completions.create(
                model='gpt-4o-2024-05-13',
                messages=[{'role': 'user', 'content': eval_prompt}],
                temperature=0.0
        )

        return completion

### Baseline 모델 개발

In [23]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [19]:
reviews, _ = preprocess_reviews(path='./resources/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

다양한 리뷰를 요약하면, 숙소는 깨끗하고 직원들이 친절하며 위치가 좋다는 공통점이 있습니다. 수영장이나 조식 등 시설에 대한 긍정적인 평가도 많이 있습니다. 몇몇 리뷰에서는 주차비가 비싸다는 언급이 있지만 전반적으로 만족스러운 경험을 한다는 내용이 많이 담겨 있습니다.


In [24]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [25]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A와 Assistant B의 답변을 비교해보면 다음과 같습니다:

1. **도움성 및 관련성**:
   - Assistant A는 리뷰의 주요 포인트를 간결하게 요약하여, 숙소의 청결, 친절한 서비스, 편리한 위치, 훌륭한 시설, 맛있는 조식 등을 강조했습니다. 이는 리뷰의 주요 내용을 잘 반영하고 있습니다.
   - Assistant B는 위치와 관련된 정보를 제공했지만, 인사동, 조계사, 경복궁 등 특정 관광지와의 접근성을 언급하여 실제 리뷰 내용과는 다소 관련이 없는 정보를 포함하고 있습니다. 이는 리뷰의 주요 포인트를 정확히 반영하지 못한 것으로 보입니다.

2. **정확성**:
   - Assistant A는 리뷰에서 언급된 주요 요소들을 정확하게 반영했습니다.
   - Assistant B는 리뷰에서 언급되지 않은 특정 관광지와의 접근성을 언급하여 정확성이 떨어집니다.

3. **깊이 및 세부사항**:
   - Assistant A는 리뷰의 주요 포인트를 간결하게 요약했지만, 깊이나 세부사항이 부족할 수 있습니다.
   - Assistant B는 특정 관광지와의 접근성을 언급했지만, 이는 리뷰의 실제 내용과는 관련이 없으므로 깊이나 세부사항이 부적절합니다.

4. **창의성**:
   - Assistant A는 창의적인 요소는 없지만, 리뷰의 주요 내용을 정확하게 반영했습니다.
   - Assistant B는 창의적으로 접근하려 했으나, 실제 리뷰 내용과 관련이 없는 정보를 포함하여 오히려 부정적인 영향을 미쳤습니다.

종합적으로 평가하면, Assistant A의 답변이 리뷰의 주요 내용을 정확하게 반영하고 있으며, 사용자에게 더 도움이 되는 정보를 제공하고 있습니다. 따라서 최종 평가 결과는 다음과 같습니다:

[[A]]


### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature를 높여서 평가 진행

In [26]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['다양한 고객들이 다양한 호평을 한다. 일부 고객들은 숙소의 청결과 서비스에 대해 만족하며, 위치와 편안한 시설 등을 칭찬한다. 어떤 고객들은 뷰에 만족하며, 어떤 고객들은 조식이 맛있거나 직원들의 친절에 감동한다. 주차비가 고가로 느껴지거나 먼지 문제 등의 불편한 점을 지적하는 고객도 있다. 이를 토대로 종합적으로 판단하면, 다양한 호평을 받는 성공적인 숙소로 요약할 수 있다.',
 '인사동점 후에 용산점 이용한 손님들 칭찬. 건물과 직원 친절, 무인락커 편리. 호텔 추천하는 친절한 여직원, 잘 쉬었다고 함. 시설 만족, 깔끔하고 고객응대 칭찬.깨끗하고 다 만족하는 숙소.',
 '용산역 주변에 위치한 숙소로 깨끗하고 친절한 직원들이 돋보였으며, 편리한 시설과 무인락커가 인상적이었습니다. 수영장은 조금 작았지만 숙박을 즐거워하는 게스트가 많이 나타났고, 전반적으로 만족스러운 경험을 한 다는 내용으로 요약할 수 있습니다.',
 '다양한 리뷰 속에서 숙소는 깨끗하고 직원들은 친절하며, 위치가 편리하며 시설이 좋다는 공통점을 확인했다. (요약에는 실제 리뷰 내용을 반영하였습니다.)',
 '다양한 고객들의 리뷰를 종합해보면, 인사동점에서 너무 좋은 경험을 하여 용산점까지 이용한 고객, 친절한 직원과 깨끗한 시설을 칭찬하는 고객, 수영장을 좋아하는 고객, 도시 전망을 즐기며 편안하게 쉬었다고 말하는 고객 등이 있습니다. 위치가 좋고 청결하며 고객 서비스가 우수한 호텔로 만족한 고객들이 많다는 것을 알 수 있습니다.',
 '다양한 고객들이 용산역 인근의 깨끗하고 친절한 숙소를 찾아 만족했다. 직원 서비스, 시설, 위치 등 다양한 요소들이 좋은 평가를 받았으며 특히 친절한 직원과 편안한 침대로 편안한 휴가를 보냈다고 이곳을 추천했다.',
 '다양한 리뷰 중에서: "용산역과 가깝고 깨끗한 객실, 친절한 직원, 수영장이 좋다"로 요약할 수 있겠네요.',
 '다양한 리뷰들을 종합하면, 용산에 위치한 나인트리 프리미어 숙소는 깨끗하고 친절한 직원들이 있으며, 가격 대비 만족스러

In [27]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_count, b_count, draw_count = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_count += 1
        elif '[[B]]' in verdict_text:
            b_count += 1
        elif '[[C]]' in verdict_text:
            c_count += 1
        else:
            print('Evaluation Error')

    return a_count, b_count, draw_count

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summaries_baseline for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:52<00:00,  5.30s/it]

Wins: 1, Losses: 9, Ties: 0


### 모델 고도화 1 - 조건들 명시

In [28]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
print(f'Reviews: {summaries}' + '\n\n')

wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

Reviews: ['이나에 위치한 숙소 인사동점을 방문한 후 용산점에서도 즐거운 시간을 보냈습니다. 건물이 매우 깨끗하고 직원들이 매우 친절합니다. 무인락커가 갖추어져 있어 짐을 보관하는 데도 편리했습니다. 정말 즐거운 경험이었습니다.\n\n용산역과 가까운 숙소로서 주변에 아이파크몰도 위치해 있어서 편리했습니다. 친구들과 시간을 보내기에 아주 좋은 환경이었습니다. 숙소는 깔끔하고 서비스는 친절했습니다. 좋은 전망을 즐길 수 있는 수영장이 독특한 매력을 지녔습니다.\n\n서울 출장을 위해 편안히 쉴 수 있는 곳이었습니다. 객실은 깨끗하고 야경은 멋있었으며, 수영장은 작았지만 재미있게 이용할 수 있었습니다. 전반적으로 만족스럽고 다시 방문할 의향이 있습니다. 위치가 편리하며 시설이 깨끗하고 친절한 서비스를 제공하여 모든 면에서 만족스러운 경험이었습니다.', '인사동점을 이용하고 너무 만족스러웠어서 용산점으로 이용했습니다. 건물과 직원 모두 깨끗하고 친절했습니다. 무인락커를 이용해서 짐을 보관하는 것도 매우 편리하고 좋았습니다. 더불어 수영장이 이쁘고 사진 찍기 좋아서 좋은 평가를 받았습니다. 감사합니다.환경도 깨끗하게 쉬다왔고 방음도 잘됐어요. 다음에 또 방문 할 계획입니다.전반적으로 좋은 숙소였습니다.재방문할 의사가 있고, 모두 만족스러운 시간을 보냈습니다. 위치도 좋고 친절한 서비스에 감사드립니다. 다음에 또 방문하겠습니다.', '인사동점 이용 후 좋아해용용산점도 마음에 들어요~\n건물이 깨끗하고 직원분들이 친절해요^^\n무인락커로 짐보관이 너무 편했습니다~\n\n수영장도 이쁘고 사진 찍기 좋아요.\n직원분들이 친절해서 좋았어요.\n\n프론트 여자직원이 정말 친절해서 인상적이에요.\n서울 방문 시 여기가 1순위에요!\n재방문 의향이 있어요.\n\n한강뷰가 너무 잘 보였어요!\n통창으로 매우 만족했습니다.\n재방문 의사 있어요.\n\n전반적으로 만족스럽고 좋았어요.\n친절한 직원분들에게 매력을 느꼈습니다.', '서울에서 체크인했을 때, 숙소는 세련된 분위기의 깨

100%|██████████| 10/10 [00:41<00:00,  4.19s/it]

Wins: 10, Losses: 0, Ties: 0


### 모델 고도화 2 - 입력 데이터의 품질 평가

In [42]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./resources/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filterd_count = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date # 어차피 6개월 안에 들어와서..

        if review_date < date_boundary:
            continue

        if len(r['review']) < 30: # 30글자 이하 리뷰 필터링
            filterd_count += 1
            print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    print(f'Good Reviews {len(reviews_good)}, Filtered: {filterd_count}')

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

수영장이 이쁘고 사진찍기 좋아요. 친절하셔서 좋앗어요
깔끔하게 쉬다 옴니다요
서울 출장으로 편하게 쉬었습니다
전반적으로 만족스럽고 좋았습니다
방은 나무랄데 없이 깔끔했고 전망도 좋았어요.
깨끗하고 좋앗어요
다음에 또가고 싶어요
너무 최고였고 좋은 시간 보냈습니당 좋아용
좋아요~~다음에 재방문할게요
항상 편히 잘쉬다 갑니다
미국식인테리어

세련되지만 그곳에 편안함이 있다.
너무친절해서 감동했습니다.
편하게 잘 쉬었어요
시설도 좋고 깔끔해서 잘 쉬다 갑니다!!
전반적으로 다 좋았습니다~
전반적으로 청결하고 직원분들이 친절해요
제가 다닌 호텔중에서 단연 최고라 할수 있습니다
너무 만족스러운 하루 였어요
하루지만 행복했습니다!
좋은 추억 남기고갑니다ㅎㅎ
편히 쉬다 갑니다.
너무 친절하십니다 잘 쉬다갑니다!!
스위트룸이라 넓고 좋았어요
편하고 좋았습니다
다음에 또 방문할게요
담배 냄새가 나서 불쾌했음
침대가 너무 폭신해서 꿀잠 잤습니다!
즐겁고 편안한 시간 보냈어요
너무 깔끔하고 좋은 가격에 호캉스 잘 즐기고 갑니다!
객실 넓고 깨끗하고 좋았어요 위치도 좋습니다
깨끗하고 넓고 좋네요
역근처이고 깔끔하고 좋았어요~
새로생긴 곳이라 깨끗한 시설이 최대 장점이에요
너무 잘 쉬었다 가요!
용산역에서 가깝고 한경뷰가 너무 머싯었어요
쾌적하고 좋았습니다.
깨끗하고 좋았습니다 ㅎㅎ
잘 쉬었다 갑니다.
너무너무 깨끗하고 포근하고 좋았어요
너무너무 즐거운 시간이었습니다ㅎㅎ
용산역 앞이고.좋네요
얼음을 가져갈수 있는 층이 있었음 하네요
너무 좋았습니다 매우 추천합니다!
최곱니다 정말로 너무 좋았어요
좋아요. 잘사용했습니다.
좁지만 좋은 호텔입니다
Good Reviews 40, Filtered: 43


In [31]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:39<00:00,  3.97s/it]

Wins: 10, Losses: 0, Ties: 0


모델 고도화 3 - Few-Shot Prompting

In [37]:
reviews_1shot, _ = preprocess_reviews('./resources/ninetree_pangyo_reviews.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

여기는 내 최애 늘 좋다 깔끔하고 쉰거같다
깡끔하고 편안해서 너무 좋았습니다!
침구류 깔끔하고 좋아요!!!
친절하고 깨끗해요.
아늑하고 인프라가 잘 갖춰져서 좋아요
늘 깨끗하고 친절한 곳입니다
좋어욪ㅇㅇㅈ아훃좋으
편하게 잘사용했습니다
좋았어요!! 재방문하고 싶어요~
너무좋아요 진짜 쉰거같아요
쾌적하게 잘 쉬고 왔습니다.
너무 편하게 쉬고 갑니다!
고객응대가 좋아요.
침구 깨끗해요.
5성급 호텔만큼 침구가 참 좋았습니다.
깨끗하고 몰이랑 붙어있어서 편리하고 좋았습니다!
프런트가 좀 친절하기를~~
깨끗하고 방이랑 침대가 넓어서 좋아요
깨끗하고 넓어서 좋아요 
다만 방음이 안됩니다.
자주이용하는곳입니다 편안하고 조아요
always very good
좋아요 좋아요 좋아요
위치 가성비 시설 만족스러워요!
스위트 방 넓어서 좋네요 ㅎㅎ
좋아요 좋아요 좋아요
깔끔하고 쉬기 좋았어요~!
다음에 또 오고싶은 호텔
잘쉬다갑니다
다음에또갈게요
수영장과 욕조때문에 방문했는데 만족했어요
청결하고 친절한 곳입니다
주차 편하고 깔끔합니다
깨끗하고 좋앗어요!!
깨끗해서 좋앗어요.
시설도 깔끔하고 주변 식당도많아 좋았습니다
가성비가 정말 훌륭합니다.
늘 잘 애용하고있습니다
방이 넓고 깨끗해서 좋아요
이번에도 편안히 잘 쉬었습니다
수영장 너무 좋고 컨디션 너무 조아요
전체적으로 깔끔하고 조용히 쉬기좋았습니다
아이와 좋은시간 보내다 왔네요~~
수영장이 좋고 한적한 곳에 있어서 좋네요
빠르게 체크인도 가능하고 주차도 편리해요
이번에도 잘 쉬었다 갑니다. 감사합니다.
부대시설이 잘 되어있어 편하고 좋았습니다!
4성급 호텔인데 위치때문인지 가성비가 좋아요
수영장은 최고 방음은 살짝 아쉬워요
넉넉한 시간과 좋은 컨디션 굿
출장때문에 10박 연박중인데 좋습니다~
방도 널찍하고 깨끗했어요. 수영장도 좋았습니다
넬리베이터 두번 갈아타는거 빼고는 편합니다.
좋아요 호캉스하기에 딱이에요
수영장도 있고 좋습니다.
전반적으로 쾌적. 호텔 자체. 규모느 작음
주차가 너무 편하고 주변에 쇼핑몰이 있어서 좋아요
주차가

KeyboardInterrupt: 

In [33]:
summaries

['요즘 많은 고객들이 인사동점에서 너무 좋았다는 후기를 받아 유산점이용했습니다. 건물이 깨끗하고 직원들이 매우 친절해서 만족스러웠어요. 무인락커가 있어서 짐 보관이 편했고 좋은 경험이었습니다. 또한, 용산역과 가까워서 주변에 있어 머물기 편했으며, 조식이 맛있고 다양했습니다. 수영장이 작았지만 잘 즐겼고, 친절한 서비스와 함께 다음 방문을 기대할 만큼 좋았습니다. 위치도 좋고 청결하며 편안하고, 높은 퀄리티의 서비스로 만족스러웠던 호텔이었습니다.',
 '숙소는 깨끗하고 직원들은 매우 친절합니다. 게다가 무인락커가 있어서 짐 보관도 편리했습니다. 체크인할 때 여자직원이 매우 친절했고, 다음 방문시 우선순위로 고려 중이라는 평이 있습니다. 숙소는 용산역과 가깝고 아이파크몰 주변에 위치해 있어 여성 3명이 머물기에도 편리합니다. 방의 전망과 수영장 크기에 대한 다양한 평가가 있었지만 전반적으로 숙박 경험이 만족스러웠다고 평가되었습니다. 호텔은 깨끗하고 친절하며 좋은 위치의 장점을 지녔다는 평이 있습니다.',
 '숙소는 깨끗하고 직원들은 친절했습니다. 무인락커로 짐 보관이 편리하고, 재방문 의사가 있는 곳입니다. 방음도 괜찮고, 베개 높이가 딱 맞아 여자 3명이 머물기에 편한 숙소입니다. 한강뷰는 통창으로 잘 보여서 만족스럽고, 야경도 훌륭하며 수영장에서는 재밌게 놀 수 있었습니다. 전반적으로 깨끗하고 위치가 좋아 재방문할 의향이 있는 숙소입니다. 위치는 용산역 바로 옆에 있어 교통이 편리하고, 친절한 서비스로 흠 잡을 곳 없는 만족스러운 숙박이었습니다.',
 '이용한 인사동점이 너무 좋아서 용산점을 이용했는데, 건물이 깨끗하고 직원들이 매우 친절해서 만족스러웠습니다. 무인락커가 있어서 짐보관도 편리했어요. 용산역과 가깝고 아이파크몰도 인접하여 쇼핑하기에도 좋았고, 베개 높이도 적절하고 여성 3명이 머물기에도 편리한 공간이었습니다. 야경이 훌륭하고, 전망이 좋은 방에서 아침을 즐기며 수영장도 즐기기에 적당했습니다. 깨끗하고 친절한 서비스로 만족하고 다음 방문을 고려

In [34]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:41<00:00,  4.12s/it]

Wins: 10, Losses: 0, Ties: 0


In [35]:
summaries

['이 호텔은 깨끗하고 친절한 직원들이 인상적이며, 용산역과의 근접성도 좋다는 평이 많습니다. 객실은 깔끔하고 청결하며, 수영장은 작지만 즐거운 시간을 보낼 수 있다고 언급되어 있습니다. 조식은 매우 맛있어서 훌륭하다는 의견도 있습니다. 일부 투숙객은 욕실 세면대 부분의 청결에 대해 조금 더 신경 써야 한다는 의견을 남겼습니다. 종합적으로 만족도가 높은 호텔로, 재방문 의사가 높은 편이라고 평가됩니다.',
 '위의 숙소 리뷰들을 종합하면, 나인트리 프리미어 용산은 깔끔하고 청결한 시설을 갖추고 있어 만족도가 높습니다. 위치가 용산역과 가깝고 주변에 상업시설이 풍부하여 편리한 점이 좋은 평가를 받았습니다. 또한, 직원들의 친절함과 수영장의 쾌적함, 그리고 맛있는 조식으로 편안한 숙박을 경험할 수 있다는 점이 강조되었습니다. 물음표키드라이기등 몇몇 불편한 점은 있지만, 전반적으로 재방문 의사가 높은 호텔로 추천되었습니다.',
 '이 숙소는 소통의 첫발자국인 만큼 직원들의 친절함과 최상의 서비스가 인상적이라고 평가받고 있습니다. 객실은 깨끗하고 넓으며 화장실도 넓은 편으로 소비자들에게 호평을 받고 있습니다. 또한 위치가 용산역과 가까워 교통 접근성도 높다는 장점을 가지고 있습니다. 하지만 몇몇 리뷰에서는 샤워실 등 청결 상태에 조금 신경 써야 한다는 언급이 있었습니다. 종합적으로 보았을 때, 다양한 서비스와 숙소의 만족도가 높아 재방문 의사가 높은 숙소로 평가됩니다.',
 '이 곳은 깔끔한 시설과 친절한 서비스로 인기를 끌고 있는 호텔로, 용산역과 인접한 위치에 있어 접근성이 좋다는 평이 많습니다. 특히 청결한 객실, 친절한 직원, 그리고 맛있는 조식 등이 만족도를 높이는 요인으로 언급되고 있습니다. 다만 몇몇 리뷰에서는 먼지가 조금 돌아다닌다는 언급이 있었지만, 전반적으로 재방문 의사가 높은 호텔로 손색이 없는 호텔로 평가받고 있습니다.',
 '이 숙소는 청결하고 친절한 직원이 인상적이며, 용산역과의 가까운 위치가 장점으로 손꼽힙니다. 수영장은 작지만 인스타갬성을

In [43]:
reviews_1shot, _ = preprocess_reviews('./resources/ninetree_pangyo_reviews.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

여기는 내 최애 늘 좋다 깔끔하고 쉰거같다
깡끔하고 편안해서 너무 좋았습니다!
침구류 깔끔하고 좋아요!!!
친절하고 깨끗해요.
아늑하고 인프라가 잘 갖춰져서 좋아요
늘 깨끗하고 친절한 곳입니다
좋어욪ㅇㅇㅈ아훃좋으
편하게 잘사용했습니다
좋았어요!! 재방문하고 싶어요~
너무좋아요 진짜 쉰거같아요
쾌적하게 잘 쉬고 왔습니다.
너무 편하게 쉬고 갑니다!
고객응대가 좋아요.
침구 깨끗해요.
5성급 호텔만큼 침구가 참 좋았습니다.
깨끗하고 몰이랑 붙어있어서 편리하고 좋았습니다!
프런트가 좀 친절하기를~~
깨끗하고 방이랑 침대가 넓어서 좋아요
깨끗하고 넓어서 좋아요 
다만 방음이 안됩니다.
자주이용하는곳입니다 편안하고 조아요
always very good
좋아요 좋아요 좋아요
위치 가성비 시설 만족스러워요!
스위트 방 넓어서 좋네요 ㅎㅎ
좋아요 좋아요 좋아요
깔끔하고 쉬기 좋았어요~!
다음에 또 오고싶은 호텔
잘쉬다갑니다
다음에또갈게요
수영장과 욕조때문에 방문했는데 만족했어요
청결하고 친절한 곳입니다
주차 편하고 깔끔합니다
깨끗하고 좋앗어요!!
깨끗해서 좋앗어요.
시설도 깔끔하고 주변 식당도많아 좋았습니다
가성비가 정말 훌륭합니다.
늘 잘 애용하고있습니다
방이 넓고 깨끗해서 좋아요
이번에도 편안히 잘 쉬었습니다
수영장 너무 좋고 컨디션 너무 조아요
전체적으로 깔끔하고 조용히 쉬기좋았습니다
아이와 좋은시간 보내다 왔네요~~
수영장이 좋고 한적한 곳에 있어서 좋네요
빠르게 체크인도 가능하고 주차도 편리해요
이번에도 잘 쉬었다 갑니다. 감사합니다.
부대시설이 잘 되어있어 편하고 좋았습니다!
4성급 호텔인데 위치때문인지 가성비가 좋아요
수영장은 최고 방음은 살짝 아쉬워요
넉넉한 시간과 좋은 컨디션 굿
출장때문에 10박 연박중인데 좋습니다~
방도 널찍하고 깨끗했어요. 수영장도 좋았습니다
넬리베이터 두번 갈아타는거 빼고는 편합니다.
좋아요 호캉스하기에 딱이에요
수영장도 있고 좋습니다.
전반적으로 쾌적. 호텔 자체. 규모느 작음
주차가 너무 편하고 주변에 쇼핑몰이 있어서 좋아요
주차가

In [44]:
reviews_2shot, _ = preprocess_reviews('./resources/ninetree_youngsan_reviews.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

수영장이 이쁘고 사진찍기 좋아요. 친절하셔서 좋앗어요
깔끔하게 쉬다 옴니다요
서울 출장으로 편하게 쉬었습니다
전반적으로 만족스럽고 좋았습니다
방은 나무랄데 없이 깔끔했고 전망도 좋았어요.
깨끗하고 좋앗어요
다음에 또가고 싶어요
너무 최고였고 좋은 시간 보냈습니당 좋아용
좋아요~~다음에 재방문할게요
항상 편히 잘쉬다 갑니다
미국식인테리어

세련되지만 그곳에 편안함이 있다.
너무친절해서 감동했습니다.
편하게 잘 쉬었어요
시설도 좋고 깔끔해서 잘 쉬다 갑니다!!
전반적으로 다 좋았습니다~
전반적으로 청결하고 직원분들이 친절해요
제가 다닌 호텔중에서 단연 최고라 할수 있습니다
너무 만족스러운 하루 였어요
하루지만 행복했습니다!
좋은 추억 남기고갑니다ㅎㅎ
편히 쉬다 갑니다.
너무 친절하십니다 잘 쉬다갑니다!!
스위트룸이라 넓고 좋았어요
편하고 좋았습니다
다음에 또 방문할게요
담배 냄새가 나서 불쾌했음
침대가 너무 폭신해서 꿀잠 잤습니다!
즐겁고 편안한 시간 보냈어요
너무 깔끔하고 좋은 가격에 호캉스 잘 즐기고 갑니다!
객실 넓고 깨끗하고 좋았어요 위치도 좋습니다
깨끗하고 넓고 좋네요
역근처이고 깔끔하고 좋았어요~
새로생긴 곳이라 깨끗한 시설이 최대 장점이에요
너무 잘 쉬었다 가요!
용산역에서 가깝고 한경뷰가 너무 머싯었어요
쾌적하고 좋았습니다.
깨끗하고 좋았습니다 ㅎㅎ
잘 쉬었다 갑니다.
너무너무 깨끗하고 포근하고 좋았어요
너무너무 즐거운 시간이었습니다ㅎㅎ
용산역 앞이고.좋네요
얼음을 가져갈수 있는 층이 있었음 하네요
너무 좋았습니다 매우 추천합니다!
최곱니다 정말로 너무 좋았어요
좋아요. 잘사용했습니다.
좁지만 좋은 호텔입니다
Good Reviews 44, Filtered: 43


100%|██████████| 10/10 [00:44<00:00,  4.43s/it]

Wins: 9, Losses: 1, Ties: 0
